In [ ]:
import torch
import torch_geometric
import torch_scatter
import torch_sparse
import torch_cluster
import torch_spline_conv

print("✅ All PyG packages imported successfully!")


In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN
from deeprobust.graph.global_attack import Metattack

data = Dataset(root='/tmp/', name='citeseer', setting='nettack')
adj, features, labels = data.adj, data.features, data.labels
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading citeseer dataset...
Selecting 1 largest connected components


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=64,
                with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

original_adj_dense = adj.copy()

/home/niyati/miniconda3/envs/ersp_v2/lib/python3.9/site-packages/deeprobust/graph/utils.py:356: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:644.)
  return torch.sparse.FloatTensor(sparseconcat.t(),sparsedata,torch.Size(sparse_mx.shape))


In [3]:
model = Metattack(model=surrogate, nnodes=adj.shape[0], feature_shape=features.shape, device=device)
model = model.to(device)
perturbations = int(2.00 * (adj.sum() // 2))
model.attack(features, adj, labels, idx_train, idx_unlabeled, perturbations, ll_constraint=False)
modified_adj = model.modified_adj

# Convert modified adjacency tensor to numpy for comparison
if isinstance(modified_adj, torch.Tensor):
    modified_adj_dense = modified_adj.cpu().numpy()
else:
    modified_adj_dense = modified_adj.todense()


# Find added and removed edges
added_edges = []
removed_edges = []

for i in range(adj.shape[0]):
    for j in range(adj.shape[0]):
        # Check if edge was added (wasn't in original but is in modified)
        if original_adj_dense[i, j] == 0 and modified_adj_dense[i, j] != 0:
            added_edges.append((i, j))
        # Check if edge was removed (was in original but not in modified)
        elif original_adj_dense[i, j] != 0 and modified_adj_dense[i, j] == 0:
            removed_edges.append((i, j))

# Combine into one list (optional)
all_perturbed_edges = added_edges + removed_edges

print(f"Added edges: {len(added_edges)}")
print(f"Removed edges: {len(removed_edges)}")
print(f"Total perturbed edges: {len(all_perturbed_edges)}")
print(f"Sample added edges: {added_edges[:5] if added_edges else 'None'}")
print(f"Sample removed edges: {removed_edges[:5] if removed_edges else 'None'}")

Perturbing graph:   0%|                                                                        | 0/7336 [00:00<?, ?it/s]

GCN loss on unlabled data: 1.0672743320465088
GCN acc on unlabled data: 0.7424960505529226
attack loss: 0.16406464576721191


Perturbing graph:   0%|                                                              | 1/7336 [00:02<4:05:44,  2.01s/it]

GCN loss on unlabled data: 1.0735424757003784
GCN acc on unlabled data: 0.7367035281727224
attack loss: 0.17865921556949615


Perturbing graph:   0%|                                                              | 2/7336 [00:03<3:16:03,  1.60s/it]

...

GCN loss on unlabled data: 15.989665985107422
GCN acc on unlabled data: 0.05687203791469194
attack loss: 8.60916519165039


Perturbing graph: 100%|██████████████████████████████████████████████████████████▉| 7335/7336 [2:48:51<00:01,  1.37s/it]

GCN loss on unlabled data: 16.174522399902344
GCN acc on unlabled data: 0.05529225908372827
attack loss: 8.69323444366455


Perturbing graph: 100%|███████████████████████████████████████████████████████████| 7336/7336 [2:48:52<00:00,  1.38s/it]


Added edges: 13832
Removed edges: 840
Total perturbed edges: 14672
Sample added edges: [(0, 648), (0, 660), (0, 1564), (0, 1797), (1, 1626)]
Sample removed edges: [(0, 253), (0, 894), (0, 1436), (14, 106), (15, 246)]


In [4]:
import os
save_path = os.path.join(os.getcwd(), "GreedySingle_Citeseer_edges_list.pth")
torch.save(all_perturbed_edges, save_path)


In [19]:
print(len(all_perturbed_edges))
print(budget)

20276
527


In [16]:
rm -rf ./Cora

In [27]:
sys.path.append(os.path.abspath("../../"))
from torch.serialization import safe_globals
from torch_geometric.data import Data
from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.greedy_gd import *


# Allow torch.load to unpickle PyG's Data object
with safe_globals([Data]):
    from torch_geometric.datasets import Planetoid
    import torch

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cora_dataset = Planetoid(root='.', name='Cora')
    data = cora_dataset[0].to(device)
    print(data)

model = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.20
budget = int(ptb_rate * initial_edge_count)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Epoch 0, Train Loss - 1.9513059854507446, Val Loss - 1.8159871101379395, Val Accuracy - 0.673
Epoch 20, Train Loss - 0.015081415884196758, Val Loss - 0.6655123233795166, Val Accuracy - 0.792
Epoch 40, Train Loss - 0.0073927463963627815, Val Loss - 0.6706228852272034, Val Accuracy - 0.791
Epoch 60, Train Loss - 0.011818666942417622, Val Loss - 0.6108243465423584, Val Accuracy - 0.81
Epoch 80, Train Loss - 0.014592904597520828, Val Loss - 0.6196398138999939, Val Accuracy - 0.806
Epoch 100, Train Loss - 0.016452601179480553, Val Loss - 0.6524885892868042, Val Accuracy - 0.798
Epoch 120, Train Loss - 0.010858620516955853, Val Loss - 0.6451842188835144, Val Accuracy - 0.804
Epoch 140, Train Loss - 0.009836899116635323, Val Loss - 0.6186845898628235, Val Accuracy - 0.816
Epoch 160, Train Loss - 0.009267968125641346, Val Loss - 0.649171769618988, Val Accuracy - 0.805
Epoch 180, Train Lo

In [30]:
from src.attacks.greedy_gd import Metattack as MyMetattack
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(all_perturbed_edges):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = all_perturbed_edges[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = MyMetattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [31]:
splits = [0, 0.5]
split_dic = defaultdict(dict)
for s in splits:
    split_dic[s] = two_phase_attack(s)


--- Running split: 0 ---
Second phase perturbation rate: 0.1000
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

RuntimeError: [enforce fail at alloc_cpu.cpp:118] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 79404582592 bytes. Error code 12 (Cannot allocate memory)